# 1. Uninformed Search

This section covers several search strategies that come under the heading of **uninformed search** (also called **blind search**). The term means that the strategies have no additional information about states beyond that provided in the problem definition. All they can do is generate successors and distinguish a goal state from a non-goal state. All search strategies are distinguished by the order in which nodes are expanded. Strategies that know whether one non-goal state is “more promising” than another are called **informed search** or **heuristic search** strategies; they are covered in Section 2.

## Breadth first search (BFS)

**Breadth-first search** is a simple strategy in which the root node is expanded first, then all the successors of the root node are expanded next, then their successors, and so on. In general, all the nodes are expanded at a given depth in the search tree before any nodes at the next level are expanded.

Breadth-first search is an instance of the general graph-search algorithm in which the shallowest unexpanded node is chosen for expansion. This is achieved very simply by using a FIFO queue for the frontier. Thus, new nodes (which are always deeper than their parents) go to the back of the queue, and old nodes, which are shallower than the new nodes, get expanded first. There is one slight tweak on the general graph-search algorithm, which is that the goal test is applied to each node when it is generated rather than when it is selected for expansion. This decision is explained below, where we discuss time complexity. Note also that the algorithm, following the general template for graph search, discards any new path to a state already in the frontier or explored set; it is easy to see that any such path must be at least as deep as the one already found. Thus, breadth-first search always has the shallowest path to every node on the frontier.

![BFS](images/bfs.png "BFS")

So far, the news about breadth-first search has been good. The news about time and space is not so good. Imagine searching a uniform tree where every state has $b$ successors. The root of the search tree generates $b$ nodes at the first level, each of which generates b more nodes, for a total of $b^2$ at the second level. Each of these generates $b$ more nodes, yielding $b^3$ nodes at the third level, and so on. Now suppose that the solution is at depth $d$. In the worst case, it is the last node generated at that level. Then the total number of nodes generated is

$b+b^2 +b^3 +···+b^d =O(b^d).$

(If the algorithm were to apply the goal test to nodes when selected for expansion, rather than when generated, the whole layer of nodes at depth d would be expanded before the goal was detected and the time complexity would be $O(b^{d+1}).$)

As for space complexity: for any kind of graph search, which stores every expanded node in the explored set, the space complexity is always within a factor of b of the time complexity. For breadth-first graph search in particular, every node generated remains in memory. There will be $O(b^{d−1})$ nodes in the explored set and $O(b^d)$ nodes in the frontier,so the space complexity is O(bd), i.e., it is dominated by the size of the frontier. Switching to a tree search would not save much space, and in a state space with many redundant paths, switching could cost a great deal of time.
An exponential complexity bound such as O(bd) is scary. Figure 3.13 shows why. It lists, for various values of the solution depth d, the time and memory required for a breadth- first search with branching factor b = 10. The table assumes that 1 million nodes can be generated per second and that a node requires 1000 bytes of storage. Many search problems fit roughly within these assumptions (give or take a factor of 100) when run on a modern personal computer. 

![BFS Time and Memory](images/bfs_time_and_memory.png "BFS")

Two lessons can be learned from Figure 3.13. First, the memory requirements are a bigger problem for breadth-first search than is the execution time. One might wait 13 days for the solution to an important problem with search depth 12, but no personal computer has the petabyte of memory it would take. Fortunately, other strategies require less memory.
The second lesson is that time is still a major factor. If your problem has a solution at depth 16, then (given our assumptions) it will take about 350 years for breadth-first search (or indeed any uninformed search) to find it. In general, exponential-complexity search problems cannot be solved by uninformed methods for any but the smallest instances.

In [133]:
def BFS(graph, start_v):
    queue = []
    visited = [start_v]
    queue.append(start_v)
    while queue:
        v = queue.pop(0)
        for w in graph[v]:
            if w not in visited:
                queue.append(w)
                visited.append(w)
    return visited

In [134]:
graph = {'A': ['B', 'C'],
         'B': ['A', 'D', 'E'],
         'C': ['A', 'F', 'G'],
         'D': ['B'],
         'E': ['B'],
         'F': ['C'],
         'G': ['C']}

ans = BFS(graph, 'A')
print(ans)

['A', 'B', 'C', 'D', 'E', 'F', 'G']


## Depth first search (DFS)

In [130]:
def DFS(graph, start_v):
    stack = []
    visited = []
    stack.append(start_v)
    while stack:
        v = stack.pop()
        if v not in visited:
            visited.append(v)
            for w in graph[v]:
                stack.append(w)
    return visited

In [132]:
graph = {'A': ['C', 'B'],
         'B': ['A', 'E', 'D'],
         'C': ['A', 'G', 'F'],
         'D': ['B'],
         'E': ['B'],
         'F': ['C'],
         'G': ['C']}

ans = DFS(graph, 'A')
print(ans)

['A', 'B', 'D', 'E', 'C', 'F', 'G']


## Uniform cost search (UCS)


In [108]:
import copy
import queue

class Graph:
    # graph dictonary contains all the adjacent nodes of each as key and value pair
    # cost_dict contains cost of each edge traversal of (u,v)
    # final_dict contains all the possible paths from start node s to goal node g with total cost
    def __init__(self):
        self.graph = dict()
        self.cost_dict = dict()
        self.final_dict = dict()

    # u and v are nodes: edge u-->v with cost c 
    def addEdge(self,u,v,c):
        if u not in self.graph:
            qu = queue.PriorityQueue()
            self.graph.update({u:qu})

        self.graph[u].put(v)
        self.cost_dict.update({(u,v):c})

    # Makes a list to keep track of visited nodes
    def tnode(self,n):
        self.visited = [False]*n
    
    def UCS_util(self,s,visited,path,goal,value):
        # Appending node to the current path 
        path.append(s)
        # Marking that node is visited 
        visited[s]=True
    
        # If goal node is reached save the path and return
        if goal==s:
            self.final_dict.update({tuple(path):value})
            return
    
        # Checking if the adjacent node is been visited and explore the new path if haven't
        for i in self.graph[s].queue:
            if visited[i]==False:
                # When new path is being explored add the cost of that path to cost of entire course traversal
                # Send a copy of path list to avoid sending it by reference
                self.UCS_util(i,copy.deepcopy(visited),copy.deepcopy(path),goal,value + self.cost_dict[s,i])

    def UCS(self, s,goal):
        self.visited[s] = True
        # List to hold all the nodes visited in path from start node to goal node 
        path=[s]
        
        for i in self.graph[s].queue:
            if self.visited[i] == False:
                # Make a variable to hold the cost of traversal
                value = self.cost_dict[s,i]
                self.UCS_util(i,copy.deepcopy(self.visited),copy.deepcopy(path),goal,value)

    # Display all the paths that is been discovered from start node to Goal node
    def all_paths(self):
        # Check if there is any path
        if bool(self.final_dict):
            print("All the paths: ")
            for i in self.final_dict:
                print("path: ",i,"cost: ",self.final_dict[i])
        else:
            print("No Path exist between start and goal node")

    # Find the most optimal path between start node to goal node
    def optimal_path(self):
        if bool(self.final_dict):
            print("best path: ",min(self.final_dict, key=self.final_dict.get))
        else:
            print("No Path exist between start and goal node")

In [109]:
g = Graph()
g.tnode(10)

g.addEdge(0, 1, 1); g.addEdge(0, 2, 1); g.addEdge(1, 3, 3);
g.addEdge(2, 5, 2); g.addEdge(3, 6, 4); g.addEdge(3, 5, 2);
g.addEdge(4, 6, 1); g.addEdge(4, 7, 5); g.addEdge(5, 4, 4);
g.addEdge(6, 7, 1); g.addEdge(5, 0, 3); g.addEdge(5, 8, 1);
g.addEdge(8, 4, 1); g.addEdge(8, 9, 3); g.addEdge(9, 7, 1);

g.UCS(0,7)
g.all_paths()
g.optimal_path()

All the paths: 
path:  (0, 1, 3, 5, 4, 6, 7) cost:  12
path:  (0, 1, 3, 5, 4, 7) cost:  15
path:  (0, 1, 3, 5, 8, 4, 6, 7) cost:  10
path:  (0, 1, 3, 5, 8, 4, 7) cost:  13
path:  (0, 1, 3, 5, 8, 9, 7) cost:  11
path:  (0, 1, 3, 6, 7) cost:  9
path:  (0, 2, 5, 4, 6, 7) cost:  9
path:  (0, 2, 5, 4, 7) cost:  12
path:  (0, 2, 5, 8, 4, 6, 7) cost:  7
path:  (0, 2, 5, 8, 4, 7) cost:  10
path:  (0, 2, 5, 8, 9, 7) cost:  8
best path:  (0, 2, 5, 8, 4, 6, 7)
